In [2]:
# Generic Comment
#* Important Comment
#! Warning
#? Suggestion
#TODO Todo list

import time
start_time = time.time()
import cyipopt
import math
import matplotlib.pyplot as plt
import numpy as np
from cyipopt import minimize_ipopt
from phe import paillier
from numba import *
%matplotlib widget

def compute_index(indices, dims):
    if len(indices) == 1:
        return indices[0]
    else:
        current_index = indices[0]
        remaining_dims_product = 1
        for dim in dims[1:]:
            remaining_dims_product *= dim
        return current_index * remaining_dims_product + compute_index(indices[1:], dims[1:])
        
end_time = time.time()
total_runtime = end_time - start_time
print("Total time required to import Modules = %f seconds " % total_runtime)

Total time required to import Modules = 11.240335 seconds 


In [3]:
start_time = time.time()

public_key, private_key = paillier.generate_paillier_keypair(n_length=256)
d1=12
def encrypt(val):
    return public_key.encrypt(val,precision = 2**(-d1))
def decrypt(val):
    return private_key.decrypt(val)

end_time = time.time()
total_runtime = end_time - start_time
print("Total time required to initialize Encryption dependencies = %f seconds " % total_runtime)

Total time required to initialize Encryption dependencies = 0.003551 seconds 


In [4]:
start_time = time.time()
######### Simulation time #########################

hc = 1e-4  # hr Numerical Integration Step Size
tFinal = 5  # hr Total Simulation Time(including multiple runs)

###################################### MPC simulation constants ##############################################

netTimeSteps = int( tFinal / hc)  # Number of calculated steps


controlStepSize   = 10 # Integration Steps per MPC Control Step
setpointStepSize  = 2 # Integration Steps per Set-Point        
PIcontrolStepSize = 1   # Integration Steps per PI Control Step

NUM_MPC_ITERATION = int(netTimeSteps/controlStepSize)  # Number of times simulation is run
HORIZON_LENGTH = 7

netControlTimeSteps   = int( netTimeSteps / controlStepSize   )
netSetPointTimeSteps  = int( netTimeSteps / setpointStepSize  )
netPIControlTimeSteps = int( netTimeSteps / PIcontrolStepSize )

controlTimeStepsPerMPC   = int( HORIZON_LENGTH * netControlTimeSteps   / NUM_MPC_ITERATION )
setpointTimeStepsPerMPC  = int( HORIZON_LENGTH * netSetPointTimeSteps  / NUM_MPC_ITERATION )
PIControlTimeStepsPerMPC = int( HORIZON_LENGTH * netPIControlTimeSteps / NUM_MPC_ITERATION )
timeStepsPerMPC          = int( HORIZON_LENGTH * netTimeSteps          / NUM_MPC_ITERATION )

controlTimeStepsPerControlAction   = int( netControlTimeSteps   / NUM_MPC_ITERATION )
setpointTimeStepsPerControlAction  = int( netSetPointTimeSteps  / NUM_MPC_ITERATION )
PIControlTimeStepsPerControlAction = int( netPIControlTimeSteps / NUM_MPC_ITERATION )
timeStepsPerControlAction          = int( netTimeSteps          / NUM_MPC_ITERATION )

NUM_OUTPUTS = 2  # Model Outputs: x1 x2
NUM_INPUTS = 4  # Model  Inputs: u1 u2 x1 x2
NUM_U = int(NUM_INPUTS - NUM_OUTPUTS)  # Number of control inputs


NUM_MPC_INPUTS = int(NUM_U * controlTimeStepsPerMPC)  # 1 set of control inputs per Horizon
NUM_MPC_CONSTRAINTS = controlTimeStepsPerMPC  # Constraints must be satisfied at all points

delta = controlStepSize * hc  # Step size for the system model
################### P matrix #############################
# V= xTP(x) "Energy of Sys"

a = 1060
b = 22
d = 0.52

###########Q Matrices for Cost Function##############
# Qx=np.diag(np.tile([2e3, 1], int(NUM_OUTPUTS/2)))
# Qu=np.diag(np.tile([8e-13, 0.001], int(NUM_OUTPUTS/2)))
loopsPerHour = int((1.0/hc)/controlStepSize)
A1 = np.repeat(np.array([1, 0.99, 1.01, 0.98, 1.02]),loopsPerHour)
A2 = np.repeat(np.array([17, 14, 5, 7, 9]),loopsPerHour)
A3 = np.repeat(np.array([1e-8, 0.8 * 1e-8, 0.84 * 1e-8, 0.9 * 1e-8, 0.92e-8]),loopsPerHour)

end_time = time.time()
total_runtime = end_time - start_time
print(
    "Total time required to initialize Control Paramters = %f seconds " % total_runtime
)

Total time required to initialize Control Paramters = 0.000999 seconds 


In [5]:
start_time = time.time()
##################### initial values and initializations ######################

# Initial Values in Deviation Variable Form
CA = 0  # kmol/m3
T = 0  # K

setpoint_record  = np.zeros((NUM_OUTPUTS, netSetPointTimeSteps))
V_record         = np.zeros(netTimeSteps)
x_record         = np.zeros((NUM_OUTPUTS, netTimeSteps + 1))
x_real           = np.array([[CA], [T]])
x_record[:, [0]] = x_real


u_record        = np.zeros((NUM_U, netPIControlTimeSteps))
uMPC_record     = np.zeros((NUM_U, netControlTimeSteps))
#grad_f_record = []

encrypted_e1 = []
encrypted_e2 = []

message_record    = []

status_record     = np.zeros(NUM_MPC_ITERATION)
cost_record       = np.zeros(NUM_MPC_ITERATION)
constraint_record = np.zeros((controlTimeStepsPerMPC,NUM_MPC_ITERATION))

realtime_data = None
cost = 0.0

################################ Parameters ######################

# Flowrate Terms
F = 5  # m^3/hr
V = 1  # m^3

# Thermochemical Properties
rhoe_L = 1e3  # kg/m3
Cp = 0.231  # kJ/kg-K
deltaH = -1.15e4  # kJ/kmol
E = 5e4  # kJ/kmol
R = 8.314  # kJ/kmolK
k0 = 8.46e6  # m3/kmol-h

# Thermal Properties
T0 = 300  # K

###################Simplifications################################
E_over_R = -E / R  # K
Coeff1 = F / V
Coeff2 = (deltaH * k0) / (rhoe_L * Cp)
Coeff3 = V * rhoe_L * Cp
######################## steady state values ########################
# Invariant Parameters
T0s = 300  # K

# State Variables
CAs = 1.9537  # kmol/m3
Ts = 401.87  # K

# Control Variables
Qs = 0.0  # kJ/hr
CA0s = 4  # kmol/m3

end_time = time.time()
total_runtime = end_time - start_time
print("Total time required to initialize Model Paramters = %f seconds " % total_runtime)

Total time required to initialize Model Paramters = 0.002001 seconds 


In [6]:
start_time = time.time()
@njit
def model_makestep(CA, T, CA0, Q, steps=1):
    for _ in range(steps):
        exp_E_over_R_Ts__CAs = np.exp(E_over_R / (Ts)) * (CAs) ** 2
        exp_E_over_R_T__CA   = np.exp(E_over_R / (T + Ts)) * (CA + CAs) ** 2
        fun_1 = (
            Coeff1 * (CA0 - CA)
            - k0 * exp_E_over_R_T__CA 
            + k0 * exp_E_over_R_Ts__CAs 
        )
        fun_2 = (
            Coeff1 * (T0 - T0s - T)
            - Coeff2 * exp_E_over_R_T__CA
            + Coeff2 * exp_E_over_R_Ts__CAs
            + (Q) / Coeff3
        )
        CA += hc * fun_1
        T  += hc * fun_2
    return CA, T
def solvePI(CA,T,CA_setpoint,T_setpoint):
    encrypted_CA_setpoint = encrypt(CA_setpoint)
    encrypted_T_setpoint  = encrypt(T_setpoint)
    encrypted_CA          = encrypt(CA)
    encrypted_T           = encrypt(T)
    encrypted_error1      = encrypted_CA_setpoint-encrypted_CA
    encrypted_error2      = encrypted_T_setpoint-encrypted_T
    encrypted_e1.append(encrypted_error1)
    encrypted_e2.append(encrypted_error2)

    Kc1    = 10
    Kc2    = 10000
    K_I1 = 20
    K_I2 = 20
    encrypted_CA0 = Kc1*encrypted_error1+K_I1*(PIcontrolStepSize*hc*sum(encrypted_e1))
    encrypted_Q   = Kc2*encrypted_error2+K_I2*(PIcontrolStepSize*hc*sum(encrypted_e2))
    CA0           = decrypt(encrypted_CA0)
    Q             = decrypt(encrypted_Q)
    if (CA0   < CAO_L):
        CA0   = CAO_L
    elif(CA0  > CAO_U):
        CA0   = CAO_U
    if (Q     < Q_L):
        Q     = Q_L
    elif(Q    > Q_U):
        Q     = Q_U
    return [CA0,Q]
def solvePI2(CA,T,CA_setpoint,T_setpoint):
    encrypted_CA_setpoint = encrypt(CA_setpoint)
    encrypted_T_setpoint  = encrypt(T_setpoint)
    encrypted_CA          = encrypt(CA)
    encrypted_T           = encrypt(T)
    encrypted_error1      = encrypted_CA_setpoint-encrypted_CA
    encrypted_error2      = encrypted_T_setpoint-encrypted_T
    encrypted_e1.append(encrypted_error1)
    encrypted_e2.append(encrypted_error2)

    Kc1    = 1000
    Kc2    = 1000000
    K_I1 = 2
    K_I2 = 2
    encrypted_CA0 = Kc1*encrypted_error1+K_I1*(PIcontrolStepSize*hc*sum(encrypted_e1))
    encrypted_Q   = Kc2*encrypted_error2+K_I2*(PIcontrolStepSize*hc*sum(encrypted_e2))
    CA0           = decrypt(encrypted_CA0)
    Q             = decrypt(encrypted_Q)
    if (CA0   < CAO_L):
        CA0   = CAO_L
    elif(CA0  > CAO_U):
        CA0   = CAO_U
    if (Q     < Q_L):
        Q     = Q_L
    elif(Q    > Q_U):
        Q     = Q_U
    return [CA0,Q]
CAO_L = -3.5
CAO_U = 3.5
Q_L = -5e5
Q_U = 5e5
end_time = time.time()
total_runtime = end_time - start_time
print("Total time required to initialize MPC Constraints = %f seconds " % total_runtime)

Total time required to initialize MPC Constraints = 0.006027 seconds 


In [36]:
from casadi import *
################################ Parameters ######################

# Flowrate Terms
F = 5  # m^3/hr
V = 1  # m^3

# Thermochemical Properties
rhoe_L = 1e3  # kg/m3
Cp = 0.231  # kJ/kg-K
deltaH = -1.15e4  # kJ/kmol
E = 5e4  # kJ/kmol
R = 8.314  # kJ/kmolK
k0 = 8.46e6  # m3/kmol-h

# Thermal Properties
T0 = 300  # K

###################Simplifications################################
E_over_R = -E / R  # K
Coeff1 = F / V
Coeff2 = (deltaH * k0) / (rhoe_L * Cp)
Coeff3 = V * rhoe_L * Cp
######################## steady state values ########################
# Invariant Parameters
T0s = 300  # K

# State Variables
CAs = 1.9537  # kmol/m3
Ts = 401.87  # K

# Control Variables
Qs = 0.0  # kJ/hr
CA0s = 4  # kmol/m3

CA = MX.sym('CA')
T = MX.sym('T')
x = vertcat(CA, T)
print(x)
CA0 = MX.sym('CA0')
Q = MX.sym('Q')
u = vertcat(CA0, Q)

exp_E_over_R_Ts__CAs = exp(E_over_R / (Ts)) * (CAs) ** 2
exp_E_over_R_T__CA   = exp(E_over_R / (T + Ts)) * (CA + CAs) ** 2
dCA = (
    Coeff1 * (CA0 - CA)
    - k0 * exp_E_over_R_T__CA 
    + k0 * exp_E_over_R_Ts__CAs 
)
dT = (
    Coeff1 * (T0 - T0s - T)
    - Coeff2 * exp_E_over_R_T__CA
    + Coeff2 * exp_E_over_R_Ts__CAs
    + (Q) / Coeff3
)
ode = vertcat(dCA, dT)
hc = 1e-4
N = 20
f = Function('f', [x,u], [ode],['CA,T','CA0,Q'],['dCA,dT'])
dae = {'x': x, 'p': u, 'ode': f(x,u)}
opts = {'min_step_size':hc, 'max_step_size':hc,}
F = integrator('F', 'cvodes', dae, opts)
print(f)
F([0.0,0.0],[3.5,5e5])

vertcat(CA, T)


RuntimeError: .../casadi/core/options.cpp:244: Unknown option: N

Did you mean one of the following?
> "t0"          [OT_DOUBLE]      "[DEPRECATED] Beginning of the time horizon"
> "tf"          [OT_DOUBLE]      "[DEPRECATED] End of the time horizon"
> "jit"          [OT_BOOL]      "Use just-in-time compiler to speed up the evaluation"
> "nadj"          [OT_INT]      "Number of adjoint sensitivities to be calculated [0]"
> "nfwd"          [OT_INT]      "Number of forward sensitivities to be calculated [0]"
Use print_options() to get a full list of options.


In [ ]:
from numba import njit
import numpy as np
@njit
def model_makestep(CA, T, CA0, Q, steps=1):
    for _ in range(steps):
        exp_E_over_R_Ts__CAs = np.exp(E_over_R / (Ts)) * (CAs) ** 2
        exp_E_over_R_T__CA   = np.exp(E_over_R / (T + Ts)) * (CA + CAs) ** 2
        fun_1 = (
            Coeff1 * (CA0 - CA)
            - k0 * exp_E_over_R_T__CA 
            + k0 * exp_E_over_R_Ts__CAs 
        )
        fun_2 = (
            Coeff1 * (T0 - T0s - T)
            - Coeff2 * exp_E_over_R_T__CA
            + Coeff2 * exp_E_over_R_Ts__CAs
            + (Q) / Coeff3
        )
        CA += hc * fun_1
        T  += hc * fun_2
    return CA, T